In [2]:
import numpy as np
import docplex.cp.utils_visu as visu
from docplex.cp.model import *

/Users/johannschmidt/git/seneca/venv/lib/python3.8/site-packages/docplex/cp/__init__.py:30: RuntimeWarning: DOcplex.CP is supported by Python versions 2.7.9+, 3.4.x, to 3.7.x, not '3.8.3.final.0'. Use it at your own risk.
  warnings.warn(msg, RuntimeWarning)


In [3]:
def calc_obj(x,proc_t,return_max=True):
    #calc value of objective function
    if return_max:
        return max(np.array(np.transpose(x)).dot(np.array(proc_t)))
    return np.array(np.transpose(x)).dot(np.array(proc_t))


def solve(proc_t,due_dates,rel_dates,nb_m):
    NbHouses = 1
    Deadline =  24*60
   
    Workers = ["M"+str(i) for i in range( nb_m)]
    
    Tasks = ["T"+str(i) for i in range( proc_t.shape[0])]

    Durations = proc_t
    ReleaseDate = rel_dates
    DueDate     = due_dates

    Skills=[]
    for w in Workers:
        for t in Tasks:
            Skills.append((w,t,1))
    """
    Precedences = [("masonry","carpentry"),("masonry","plumbing"),("masonry","ceiling"),
                   ("carpentry","roofing"),("ceiling","painting"),("roofing","windows"),
                   ("roofing","facade"),("plumbing","facade"),("roofing","garden"),
                   ("plumbing","garden"),("windows","moving"),("facade","moving"),
                   ("garden","moving"),("painting","moving")
                  ]
    """
    nbWorkers = len(Workers)
    Houses = range(NbHouses)
    mdl5 = CpoModel()
    tasks = {}
    wtasks = {}
    for h in Houses:
        for i,t in enumerate(Tasks):
            tasks[(h,t)] = mdl5.interval_var(start=[0,Deadline], size=Durations[i])
        for i,s in enumerate(Skills):
            #print(DueDate[i%len(Tasks)])
            wtasks[(h,s)] = mdl5.interval_var(start=[0,DueDate[i%len(Tasks)]],optional=True)
    for h in Houses:
        for t in Tasks:
            mdl5.add( mdl5.alternative(tasks[h,t], [wtasks[h,s] for s in Skills if s[1]==t]) )
    for w in Workers:
        mdl5.add( mdl5.no_overlap([wtasks[h,s] for h in Houses for s in Skills if s[0]==w]) )
    mdl5.add(
        mdl5.minimize( 
            mdl5.max( mdl5.end_of(wtasks[h,s]) for h in Houses for s in Skills)
        )
    )
    # Solve the model
    print("\nSolving model....")
    msol5 = mdl5.solve()
    print("done")
    if msol5 is not None:
        print("Cost will be "+str( msol5.get_objective_values()[0] ))
    worker_idx = {w : i for i,w in enumerate(Workers)}
    worker_tasks = [[] for w in range(nbWorkers)]  # Tasks assigned to a given worker
    for h in Houses:
        for s in Skills:
            worker = s[0]
            wt = wtasks[(h,s)]
            worker_tasks[worker_idx[worker]].append(wt)
    sol_dict = {k: [] for k in range(nb_m)}
    
    for i,w in enumerate(Workers):
        visu.sequence(name=w)
        for k,t in enumerate(worker_tasks[worker_idx[w]]):
            wt = msol5.get_var_solution(t)
            #print(wt)
            if wt.is_present():
                sol_dict[i].append((k,wt.start,wt.end))
    for i,w in enumerate(Workers):
        sol_dict[i].sort(key=lambda tup: tup[1])
    return sol_dict,msol5.get_objective_values()[0]

In [4]:
def gen_due_date(max_proc_t,nb_m,nb_t,pt_int=True):
    proc_l=[]
    fac_vec=np.arange(np.ceil(nb_t/2),dtype=np.int64)+1
    fac_vec=np.repeat(fac_vec, 2)
    np.random.shuffle(fac_vec)
    print(fac_vec)
    
    for i in range(nb_t):
        t=np.random.normal(loc=(fac_vec[i]*max_proc_t[i]+max_proc_t[i]/2),scale=max_proc_t[i]/6)
        if pt_int:
            proc_l.append(int(t))
        else:
            proc_l.append(t)
    return np.array(proc_l)
        
    
def simulation(pt_mean,pt_std,nb_t_range,nb_m_range,nb_data,pt_int=False,return_dur=False):
    # init parameters
    # pt_mean: mean of processtime (duration of task on every machine)
    # pt_std:std of processtime 
    # number of task range f.e. [2,10] means uniform distribution between 2 and 10 tasks
    # number of machine range f.e. [2,10] means uniform distribution between 2 and 10 machines
    # nb_data = number of datasamples which should get generated
    # pt_int = (boolean) if process time is int or float (default: False)
    # return_dur = process time get's return (for statistics, default:False)
    x=[]
    y=[]
    if return_dur:
        dur=[]
    for i in range(nb_data):
        
        nb_m=np.random.randint(nb_m_range[0],nb_m_range[1])
        nb_t=np.random.randint(nb_t_range[0],nb_t_range[1])
        
        print('it:',i,'\t#m:',nb_m,'\t#t:',nb_t)
        if pt_int:
            #proc_t=np.random.normal(loc=pt_mean,scale=pt_std,size=(nb_m,nb_t))
            proc_t=np.random.normal(loc=pt_mean,scale=pt_std,size=(nb_t))
            proc_t=proc_t.astype(np.int64)
        else:
            #proc_t=np.random.normal(loc=pt_mean,scale=pt_std,size=(nb_m,nb_t))
            proc_t=np.random.normal(loc=pt_mean,scale=pt_std,size=(nb_t))
        print(proc_t)
        
        
        
        rel_dates = np.zeros(nb_t,dtype=np.int32)
        #due_dates=gen_due_date(np.amax(proc_t, axis=0),nb_m,nb_t)
        due_dates=gen_due_date(proc_t,nb_m,nb_t)
        x.append([{'pt':proc_t,'dd':due_dates}])
       
        print(due_dates)
        sol=solve(proc_t,due_dates,rel_dates,nb_m)
        y.append([sol])
        if return_dur:
            dur.append(sol[-1])
    if return_dur:
        return x,y,dur
    return x,y
   

x,y=simulation(30,7,[3,17],[2,5],10,True)
    

it: 0 	#m: 4 	#t: 8
[34 38 27 30 26 13 19 38]
[1 2 3 4 2 3 4 1]
[ 60  93  97 130  62  50  86  54]

Solving model....


CpoException: Can not execute command 'cpoptimizer -angel'. Please check availability of required executable file.

In [84]:
np.save("x_data.npy", x)
np.save("y_data.npy", y)
"""
pls remember that .np stores the dict as structured array d2.item().get('key2') 
"""
# dict keys: x "pt","dd"
# dict keys: y 0,...,n für n maschinen

"\npls remember that .np stores the dict as structured array d2.item().get('key2') \n"

In [96]:
x_load=np.load("x_data.npy",allow_pickle=True)
for i in range(x_load.shape[0]):
    print(x_load[i].item().get('pt'))

[31 30 23 20 21 26 26 34 25 29 34 33 34 26]
[28 36 35 24 37 25 34 30 44 21 30 24]
[37 31 41 34 31 19 18 42 26 39 35 24 26 22 31 21]
[34 28 40 38 25 32 26 34 33 33]
[24 30 25 28]
[22 23 32 23 24 31 28]
[29 27 22 38 27 31 16 38 30 31 23 21 34]
[28 29 38 33 36 17 37 20 39]
[28 33 30 33 37 33 34]
[28 29 23 32 45 27 32 35 26  6 33 28 23 27 29 43]
